## Task 1 (English) System \#3 - Pretrained BERT Model

### 1. Preprocessing

Read in data:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os

os.getcwd()

'/content'

In [1]:
import pandas as pd
import numpy as np
import fast_bert
import logging
import warnings
import torch

warnings.filterwarnings("ignore")
np.random.seed(42)

train = pd.read_csv('data/training.tsv', sep='\t', header=0).drop(['tweet_url', 'topic_id', 'claim'], axis=1)
dev = pd.read_csv('data/dev.tsv', sep='\t', header=0).drop(['tweet_url', 'topic_id', 'claim'], axis=1)

train.insert(0, 'index', [x for x in range(len(train))])
dev.insert(0, 'index', [x for x in range(len(dev))])

train.head()

,index,tweet_id,tweet_text,claim_worthiness
0,0,1234964653014384644,Since this will never get reported by the medi...,1
1,1,1234869939720216578,"Thanks, #MichaelBloomberg. Here’s a handy litt...",0
2,2,1234873136304267267,"Folks, when you say ""The corona virus isn't a ...",0
3,3,1235071285027147776,Just 1 case of Corona Virus in India and peop...,1
4,4,1234911110861594624,President @realDonaldTrump made a commitment...,1


Format data for insertion into train/val .csv files for BERT model.

In [2]:
train_bert = train.drop(['tweet_id'], axis=1)
val_bert = dev.drop(['tweet_id'], axis=1)

train_bert.columns = ['index', 'text', 'label']
val_bert.columns = ['index', 'text', 'label']

train_bert['label'] = np.where(train_bert['label'] == 1, 'pos', 'neg')
val_bert['label'] = np.where(val_bert['label'] == 1, 'pos', 'neg')

train_bert.to_csv('bert_data/train.csv', index=False)
val_bert.to_csv('bert_data/val.csv', index=False)

train_bert.head()

,index,text,label
0,0,Since this will never get reported by the medi...,pos
1,1,"Thanks, #MichaelBloomberg. Here’s a handy litt...",neg
2,2,"Folks, when you say ""The corona virus isn't a ...",neg
3,3,Just 1 case of Corona Virus in India and peop...,pos
4,4,President @realDonaldTrump made a commitment...,pos


### 2. Modelling

Use fast-bert library to implement BERT-based classification model.

In [1]:
torch.cuda.empty_cache()

databunch = fast_bert.data_cls.BertDataBunch('bert_data', 'bert_data',
                                              tokenizer='bert-base-uncased',
                                              train_file='train.csv',
                                              val_file='val.csv',
                                              label_file='labels.csv',
                                              text_col='text',
                                              label_col='label',
                                              batch_size_per_gpu=16,
                                              max_seq_length=16,
                                              multi_gpu=False,
                                              multi_label=False,
                                              model_type='bert')

NameError: ignored

In [0]:
metrics = [{'name': 'F1', 'function': fast_bert.metrics.F1}]
logger = logging.getLogger()

learner = fast_bert.learner_cls.BertLearner.from_pretrained_model(databunch,
                                                                    pretrained_path='bert-base-uncased',
                                                                    metrics=metrics,
                                                                    device=torch.device('cuda'),
                                                                    logger=logger,
                                                                    output_dir='bert_data',
                                                                    finetuned_wgts_path=None,
                                                                    warmup_steps=500,
                                                                    multi_gpu=False,
                                                                    is_fp16=False,
                                                                    multi_label=False,
                                                                    logging_steps=50)

In [5]:
learner.fit(epochs=4,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")

learner.save_model()

RuntimeError: ignored

### 3. Prediction

Using the trained model from above to predict check-worthiness.